In [4]:
import os
import torch
from torch import nn

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import datasets # CIFAR10, MNIST
import pytorch_lightning as pl

import matplotlib.pyplot as plt

In [7]:
import numpy as np


class XORDataset(Dataset):
    """XOR dataset."""

    def __init__(self, num_samples, transform=None):
        """
        Args:
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.num_samples = num_samples
        self.X = np.random.randint(0,high=2, size=num_samples*2).reshape((num_samples, 2))
        self.y = np.logical_xor(self.X[:,0], self.X[:,1]).astype(np.uint)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = {'X': self.X[idx], 'y': self.y[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [11]:
X = np.random.randint(0,high=2, size=10*2).reshape((10, 2))
y = np.logical_xor(X[:,0], X[:,1]).astype(np.uint)
print(X)
print(y)
print(X[2])
print(y[2])

[[1 0]
 [0 1]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [0 1]]
[1 1 0 1 0 0 1 0 0 1]
[0 0]
0


In [5]:
class XORnet(nn.Module):

    def __init__(self):
        super(XORnet, self).__init__()
        self.linear = nn.Linear(2, 2)
        self.Sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(2, 1)
        self.layers = nn.Sequential(
            self.linear, 
            self.Sigmoid, 
            self.linear2
        )
        # custom weight initialization
        nnLinearLayers = [layer for layer in self.layers if type(layer) == nn.Linear]
        for layer in nnLinearLayers:
            nn.init.normal_(layer.weight)

    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

In [6]:
loss_fn = nn.MSELoss() 

model = XORnet()

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
epochs=10

for epoch in range(1, epochs+1): ## run the model for 11 epochs
    train_loss, valid_loss = [], []
    ## training part 
    ##--------------
    model.train()
    for data, target in loaders['train']:
        optimizer.zero_grad()
        ## 1. forward propagation
        output = model(data)
        
        ## 2. loss calculation
        loss = loss_fn(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part
    ## ---------------

    with torch.no_grad():  #gradients should not evaluate
        model.eval()
        for data, target in loaders['valid']:
            output = model(data)
            loss = lossFunction(output, target)
            valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))